# Benchmarking & Volumetric Testing

This notebook demonstrates performance benchmarking and volumetric testing capabilities of iso8583sim.

## Setup

In [1]:
import statistics
import sys
import time
from collections.abc import Callable

sys.path.insert(0, "..")

from iso8583sim.core.builder import ISO8583Builder
from iso8583sim.core.parser import ISO8583Parser
from iso8583sim.core.pool import MessagePool
from iso8583sim.core.types import ISO8583Message
from iso8583sim.core.validator import ISO8583Validator

## Benchmark Utilities

Helper functions for running benchmarks:

In [2]:
def benchmark(func: Callable, iterations: int = 5, warmup: int = 1) -> dict:
    """Run a benchmark function multiple times and return statistics.

    Args:
        func: Function that returns (count, elapsed_time)
        iterations: Number of benchmark iterations
        warmup: Number of warmup iterations (not counted)

    Returns:
        Dictionary with mean, min, max TPS and timing info
    """
    # Warmup
    for _ in range(warmup):
        func()

    # Benchmark
    results = []
    for _ in range(iterations):
        count, elapsed = func()
        tps = count / elapsed
        results.append(tps)

    return {
        "mean_tps": statistics.mean(results),
        "min_tps": min(results),
        "max_tps": max(results),
        "std_dev": statistics.stdev(results) if len(results) > 1 else 0,
        "iterations": iterations,
    }


def print_benchmark_result(name: str, result: dict) -> None:
    """Pretty print benchmark results."""
    print(f"{name}:")
    print(f"  Mean: {result['mean_tps']:>10,.0f} TPS")
    print(f"  Min:  {result['min_tps']:>10,.0f} TPS")
    print(f"  Max:  {result['max_tps']:>10,.0f} TPS")
    print(f"  Std:  {result['std_dev']:>10,.0f} TPS")
    print()

## Message Generation

First, let's create functions to generate test messages:

In [3]:
def generate_auth_message(i: int) -> ISO8583Message:
    """Generate an authorization request message."""
    return ISO8583Message(
        mti="0100",
        fields={
            0: "0100",
            2: f"411111111111{i % 10000:04d}",
            3: "000000",
            4: f"{(i % 100000):012d}",
            11: f"{i % 1000000:06d}",
            14: "2612",
            22: "051",
            41: "TERM0001",
            42: "MERCHANT123456 ",
            49: "840",
        },
    )


# Pre-generate some messages for parsing benchmarks
builder = ISO8583Builder()
sample_messages = [builder.build(generate_auth_message(i)) for i in range(100)]
print(f"Generated {len(sample_messages)} sample messages")
print(f"Sample message length: {len(sample_messages[0])} bytes")

Generated 100 sample messages
Sample message length: 95 bytes


## 1. Builder Benchmark

Measure message building throughput:

In [4]:
def benchmark_builder(count: int = 10000) -> tuple[int, float]:
    """Benchmark message building."""
    builder = ISO8583Builder()

    start = time.perf_counter()
    for i in range(count):
        msg = generate_auth_message(i)
        _ = builder.build(msg)  # Capture to prevent optimization
    elapsed = time.perf_counter() - start

    return count, elapsed


result = benchmark(lambda: benchmark_builder(10000))
print_benchmark_result("Message Building", result)

Message Building:
  Mean:    113,492 TPS
  Min:     107,844 TPS
  Max:     115,414 TPS
  Std:       3,184 TPS



## 2. Parser Benchmark

Measure message parsing throughput:

In [5]:
def benchmark_parser(count: int = 10000) -> tuple[int, float]:
    """Benchmark message parsing."""
    parser = ISO8583Parser()

    start = time.perf_counter()
    for i in range(count):
        raw = sample_messages[i % len(sample_messages)]
        _ = parser.parse(raw)  # Capture to prevent optimization
    elapsed = time.perf_counter() - start

    return count, elapsed


result = benchmark(lambda: benchmark_parser(10000))
print_benchmark_result("Message Parsing", result)

Message Parsing:
  Mean:    134,751 TPS
  Min:     133,333 TPS
  Max:     136,379 TPS
  Std:       1,400 TPS



## 3. Validation Benchmark

Measure validation throughput:

In [6]:
# Pre-parse messages for validation benchmark
parser = ISO8583Parser()
parsed_messages = [parser.parse(raw) for raw in sample_messages]


def benchmark_validator(count: int = 10000) -> tuple[int, float]:
    """Benchmark message validation."""
    validator = ISO8583Validator()

    start = time.perf_counter()
    for i in range(count):
        msg = parsed_messages[i % len(parsed_messages)]
        _ = validator.validate_message(msg)  # Capture to prevent optimization
    elapsed = time.perf_counter() - start

    return count, elapsed


result = benchmark(lambda: benchmark_validator(10000))
print_benchmark_result("Message Validation", result)

Message Validation:
  Mean:    306,999 TPS
  Min:     294,327 TPS
  Max:     323,326 TPS
  Std:      10,428 TPS



## 4. Full Roundtrip Benchmark

Measure complete cycle: Build → Parse → Validate

In [7]:
def benchmark_roundtrip(count: int = 10000, validate: bool = True) -> tuple[int, float]:
    """Benchmark full roundtrip."""
    builder = ISO8583Builder()
    parser = ISO8583Parser()
    validator = ISO8583Validator()

    start = time.perf_counter()
    for i in range(count):
        # Build
        msg = generate_auth_message(i)
        raw = builder.build(msg)

        # Parse
        parsed = parser.parse(raw)

        # Validate (optional)
        if validate:
            validator.validate_message(parsed)

    elapsed = time.perf_counter() - start
    return count, elapsed


print("With Validation:")
result = benchmark(lambda: benchmark_roundtrip(10000, validate=True))
print_benchmark_result("Build → Parse → Validate", result)

print("Without Validation:")
result = benchmark(lambda: benchmark_roundtrip(10000, validate=False))
print_benchmark_result("Build → Parse", result)

With Validation:


Build → Parse → Validate:
  Mean:     49,269 TPS
  Min:      48,334 TPS
  Max:      49,646 TPS
  Std:         538 TPS

Without Validation:


Build → Parse:
  Mean:     59,329 TPS
  Min:      57,125 TPS
  Max:      60,062 TPS
  Std:       1,237 TPS



## 5. Object Pooling Benchmark

Compare performance with and without object pooling:

In [8]:
def benchmark_pooled_roundtrip(count: int = 10000) -> tuple[int, float]:
    """Benchmark roundtrip with object pooling."""
    pool = MessagePool(size=100)
    builder = ISO8583Builder()
    parser = ISO8583Parser(pool=pool)
    validator = ISO8583Validator()

    start = time.perf_counter()
    for i in range(count):
        # Build
        msg = generate_auth_message(i)
        raw = builder.build(msg)

        # Parse (uses pool)
        parsed = parser.parse(raw)

        # Validate
        validator.validate_message(parsed)

        # Return to pool
        pool.release(parsed)

    elapsed = time.perf_counter() - start
    return count, elapsed


print("Standard (no pooling):")
result_standard = benchmark(lambda: benchmark_roundtrip(10000, validate=True))
print_benchmark_result("Standard Roundtrip", result_standard)

print("With Object Pooling:")
result_pooled = benchmark(lambda: benchmark_pooled_roundtrip(10000))
print_benchmark_result("Pooled Roundtrip", result_pooled)

improvement = ((result_pooled["mean_tps"] - result_standard["mean_tps"]) / result_standard["mean_tps"]) * 100
print(f"Pooling improvement: {improvement:+.1f}%")

Standard (no pooling):


Standard Roundtrip:
  Mean:     49,244 TPS
  Min:      47,742 TPS
  Max:      50,105 TPS
  Std:         935 TPS

With Object Pooling:


Pooled Roundtrip:
  Mean:     49,011 TPS
  Min:      47,262 TPS
  Max:      49,812 TPS
  Std:       1,028 TPS

Pooling improvement: -0.5%


## 6. Volumetric Testing

Test with different batch sizes to understand scaling behavior:

In [9]:
batch_sizes = [100, 1000, 5000, 10000, 25000, 50000]

print("Volumetric Test Results:")
print("=" * 60)
print(f"{'Batch Size':>12} | {'Build TPS':>12} | {'Parse TPS':>12} | {'Roundtrip TPS':>14}")
print("-" * 60)

for batch_size in batch_sizes:
    # Build benchmark
    _, build_time = benchmark_builder(batch_size)
    build_tps = batch_size / build_time

    # Parse benchmark
    _, parse_time = benchmark_parser(batch_size)
    parse_tps = batch_size / parse_time

    # Roundtrip benchmark
    _, roundtrip_time = benchmark_roundtrip(batch_size, validate=True)
    roundtrip_tps = batch_size / roundtrip_time

    print(f"{batch_size:>12,} | {build_tps:>12,.0f} | {parse_tps:>12,.0f} | {roundtrip_tps:>14,.0f}")

print("=" * 60)

Volumetric Test Results:
  Batch Size |    Build TPS |    Parse TPS |  Roundtrip TPS
------------------------------------------------------------
         100 |      118,407 |      141,760 |         51,138
       1,000 |      119,023 |      138,388 |         49,948


       5,000 |      114,673 |      136,107 |         49,902


      10,000 |      104,366 |      134,127 |         48,953


      25,000 |      107,758 |      134,665 |         48,667


      50,000 |      113,218 |      131,665 |         48,842


## 7. Request/Response Flow Benchmark

Simulate realistic authorization flow: Request → Parse → Create Response → Build Response

In [10]:
def benchmark_request_response(count: int = 10000) -> tuple[int, float]:
    """Benchmark request/response flow."""
    builder = ISO8583Builder()
    parser = ISO8583Parser()

    start = time.perf_counter()
    for i in range(count):
        # Build request
        request = generate_auth_message(i)
        raw_request = builder.build(request)

        # Parse request
        parsed_request = parser.parse(raw_request)

        # Create response
        response = builder.create_response(
            parsed_request,
            response_fields={
                38: "ABC123",  # Auth code
                39: "00",  # Approved
            },
        )

        # Build response
        _ = builder.build(response)  # Capture to prevent optimization

    elapsed = time.perf_counter() - start
    return count, elapsed


result = benchmark(lambda: benchmark_request_response(10000))
print_benchmark_result("Request/Response Flow", result)

Request/Response Flow:
  Mean:     27,822 TPS
  Min:      27,522 TPS
  Max:      28,048 TPS
  Std:         256 TPS



## 8. Memory Usage Analysis

Analyze memory consumption for different batch sizes:

In [11]:
import gc


def measure_memory(count: int) -> dict:
    """Measure memory usage for message operations."""
    gc.collect()

    builder = ISO8583Builder()
    parser = ISO8583Parser()

    # Build messages
    messages = []
    for i in range(count):
        msg = generate_auth_message(i)
        raw = builder.build(msg)
        messages.append(raw)

    raw_size = sum(len(m) for m in messages)

    # Parse messages (store to measure memory impact)
    _ = [parser.parse(m) for m in messages]

    return {
        "count": count,
        "raw_bytes": raw_size,
        "avg_msg_size": raw_size / count,
    }


print("Memory Analysis:")
print("-" * 50)
for count in [1000, 5000, 10000]:
    stats = measure_memory(count)
    print(f"Messages: {stats['count']:>6,}")
    print(f"  Total raw bytes: {stats['raw_bytes']:>10,}")
    print(f"  Avg message size: {stats['avg_msg_size']:.1f} bytes")
    print()

Memory Analysis:
--------------------------------------------------
Messages:  1,000
  Total raw bytes:     95,000
  Avg message size: 95.0 bytes

Messages:  5,000
  Total raw bytes:    475,000
  Avg message size: 95.0 bytes



Messages: 10,000
  Total raw bytes:    950,000
  Avg message size: 95.0 bytes



## 9. Latency Distribution

Analyze per-message latency distribution:

In [12]:
def measure_latencies(count: int = 1000) -> list[float]:
    """Measure individual message processing latencies."""
    builder = ISO8583Builder()
    parser = ISO8583Parser()
    validator = ISO8583Validator()
    latencies = []

    for i in range(count):
        start = time.perf_counter()

        msg = generate_auth_message(i)
        raw = builder.build(msg)
        parsed = parser.parse(raw)
        validator.validate_message(parsed)

        elapsed = time.perf_counter() - start
        latencies.append(elapsed * 1000)  # Convert to milliseconds

    return latencies


latencies = measure_latencies(1000)

print("Latency Distribution (per message):")
print("-" * 40)
print(f"  Min:    {min(latencies):.4f} ms")
print(f"  Max:    {max(latencies):.4f} ms")
print(f"  Mean:   {statistics.mean(latencies):.4f} ms")
print(f"  Median: {statistics.median(latencies):.4f} ms")
print(f"  Std:    {statistics.stdev(latencies):.4f} ms")

# Percentiles
sorted_latencies = sorted(latencies)
p50 = sorted_latencies[int(len(sorted_latencies) * 0.50)]
p90 = sorted_latencies[int(len(sorted_latencies) * 0.90)]
p95 = sorted_latencies[int(len(sorted_latencies) * 0.95)]
p99 = sorted_latencies[int(len(sorted_latencies) * 0.99)]

print("\nPercentiles:")
print(f"  P50:    {p50:.4f} ms")
print(f"  P90:    {p90:.4f} ms")
print(f"  P95:    {p95:.4f} ms")
print(f"  P99:    {p99:.4f} ms")

Latency Distribution (per message):
----------------------------------------
  Min:    0.0184 ms
  Max:    0.0973 ms
  Mean:   0.0202 ms
  Median: 0.0193 ms
  Std:    0.0036 ms

Percentiles:
  P50:    0.0193 ms
  P90:    0.0214 ms
  P95:    0.0217 ms
  P99:    0.0303 ms


## 10. Summary Report

Generate a comprehensive performance summary:

In [13]:
import platform

print("=" * 60)
print("ISO8583 PERFORMANCE SUMMARY")
print("=" * 60)

# Run all benchmarks
build_result = benchmark(lambda: benchmark_builder(10000))
parse_result = benchmark(lambda: benchmark_parser(10000))
validate_result = benchmark(lambda: benchmark_validator(10000))
roundtrip_result = benchmark(lambda: benchmark_roundtrip(10000, validate=True))
reqresp_result = benchmark(lambda: benchmark_request_response(10000))

print(f"\n{'Operation':<30} {'TPS':>15}")
print("-" * 50)
print(f"{'Message Building':<30} {build_result['mean_tps']:>15,.0f}")
print(f"{'Message Parsing':<30} {parse_result['mean_tps']:>15,.0f}")
print(f"{'Message Validation':<30} {validate_result['mean_tps']:>15,.0f}")
print(f"{'Full Roundtrip':<30} {roundtrip_result['mean_tps']:>15,.0f}")
print(f"{'Request/Response Flow':<30} {reqresp_result['mean_tps']:>15,.0f}")
print("-" * 50)

print("\nSystem Info:")
print(f"  Python: {platform.python_version()}")
print(f"  Platform: {platform.platform()}")
print(f"  Processor: {platform.processor()}")

ISO8583 PERFORMANCE SUMMARY



Operation                                  TPS
--------------------------------------------------
Message Building                       110,764
Message Parsing                        132,206
Message Validation                     311,669
Full Roundtrip                          48,725
Request/Response Flow                   27,829
--------------------------------------------------

System Info:
  Python: 3.12.6
  Platform: macOS-15.7.2-arm64-arm-64bit
  Processor: arm


## Performance Tips

To maximize throughput:

1. **Reuse instances** - Create parser/builder once, reuse for all messages
2. **Use object pooling** - For high-throughput scenarios with millions of messages
3. **Disable validation** - Skip validation when parsing trusted internal messages
4. **Build Cython extensions** - Run `python setup.py build_ext --inplace` for 2x speedup
5. **Disable debug logging** - Set logging level to WARNING in production

See [docs/performance.md](../docs/performance.md) for detailed optimization guidance.